# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import re
import sqlite3
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/natnareephoungsombat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/natnareephoungsombat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#load data
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM InsertDatabaseName', engine)

In [3]:
#check count of values in genre colum.  
df['genre'].value_counts()

news      12978
direct    10825
social     2404
Name: genre, dtype: int64

In [4]:
#check count of values in related colum. 
df['related'].value_counts()

1    19914
0     6101
2      192
Name: related, dtype: int64

In [5]:
#sort related is 1 and genre is not social
df = df[(df["related"] == 1) & (df['genre'] != 'social')]
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [6]:
#set data of X, y
X = df
y = df.drop(['id', 'message', 'original', 'genre', 'related'], axis=1)

### 2. Write a tokenization function to process your text data

In [7]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train['message'], y_train)

Pipeline(steps=[('cvect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd97cfaf440>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# Prediction data
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'])
    
# Optional display names matching the labels
target_names = y.columns.values

In [11]:
def classificationReport_pipeline(test, pred):  
    
    '''
    Classification metrics
        - metrics.accuracy_score(y_true, y_pred, *[, ...])
        - metrics.precision_score(y_true, y_pred, *[, ...])
        - metrics.recall_score(y_true, y_pred, *[, ...])
        - metrics.f1_score(y_true, y_pred, *[, ...])
        - metrics.classification_report(y_true, y_pred, *)
        - other...(https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
    '''
    
    # Accuracy score
    accuracy = accuracy_score(test, pred)
    
    # Precision score
    precision_weighted = precision_score(test, pred, average='weighted', zero_division=0)
    
    # Recall score
    recall_micro = recall_score(test, pred, average='micro', zero_division=0)
    
    # f1 score
    f1_macro = f1_score(test, pred, average='macro', zero_division=0)
    
    
    print('Accuracy:', accuracy)
    print('Precision weighted:', precision_weighted)
    print('Recall micro:', recall_micro)
    print('f1 macro', f1_macro)
    print(classification_report(y_true=test.values, 
                                y_pred=pred , 
                                target_names=target_names, zero_division=0))

In [12]:
classificationReport_pipeline(y_test, y_pred_test)

Accuracy: 0.13294538943598927
Precision weighted: 0.2957503013749629
Recall micro: 0.13702378446642813
f1 macro 0.041218154256780916
                        precision    recall  f1-score   support

               request       0.60      0.34      0.43      1056
                 offer       0.00      0.00      0.00        28
           aid_related       0.57      0.49      0.53      2479
          medical_help       0.20      0.00      0.00       516
      medical_products       0.33      0.00      0.01       296
     search_and_rescue       0.00      0.00      0.00       169
              security       0.00      0.00      0.00       110
              military       0.25      0.00      0.01       208
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       394
                  food       0.22      0.00      0.01       685
               shelter       0.00      0.00      0.00       558
              clothing       0.00 

In [13]:
classificationReport_pipeline(y_train, y_pred_train)

Accuracy: 0.9882853305476794
Precision weighted: 0.9979861496791761
Recall micro: 0.9918534085810763
f1 macro 0.9650390627758495
                        precision    recall  f1-score   support

               request       1.00      1.00      1.00      3217
                 offer       1.00      0.97      0.99        73
           aid_related       1.00      1.00      1.00      7630
          medical_help       1.00      0.99      0.99      1485
      medical_products       1.00      0.98      0.99       921
     search_and_rescue       1.00      0.99      1.00       493
              security       1.00      0.98      0.99       318
              military       1.00      0.99      0.99       633
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.99      0.99      1223
                  food       1.00      0.99      0.99      2044
               shelter       1.00      0.99      0.99      1664
              clothing       1.00     

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('cvect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fd97cfaf440>)),
  ('tfidf', TfidfTransformer()),
  ('clf', RandomForestClassifier())],
 'verbose': False,
 'cvect': CountVectorizer(tokenizer=<function tokenize at 0x7fd97cfaf440>),
 'tfidf': TfidfTransformer(),
 'clf': RandomForestClassifier(),
 'cvect__analyzer': 'word',
 'cvect__binary': False,
 'cvect__decode_error': 'strict',
 'cvect__dtype': numpy.int64,
 'cvect__encoding': 'utf-8',
 'cvect__input': 'content',
 'cvect__lowercase': True,
 'cvect__max_df': 1.0,
 'cvect__max_features': None,
 'cvect__min_df': 1,
 'cvect__ngram_range': (1, 1),
 'cvect__preprocessor': None,
 'cvect__stop_words': None,
 'cvect__strip_accents': None,
 'cvect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'cvect__tokenizer': <function __main__.tokenize(text)>,
 'cvect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__bootstrap': True,
 'clf

In [15]:
parameters = {
    'clf__max_depth': [5, None],
    'clf__min_samples_split': [2, 3],
    'clf__n_estimators': [5, 10, 50]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [16]:
cv.fit(X_train['message'], y_train)

GridSearchCV(estimator=Pipeline(steps=[('cvect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fd97cfaf440>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             param_grid={'clf__max_depth': [5, None],
                         'clf__min_samples_split': [2, 3],
                         'clf__n_estimators': [5, 10, 50]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
# Prediction data
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])
    
# Optional display names matching the labels
target_names = y.columns.values

In [18]:
def classificationReport_cv(test, pred):
       
    '''
    Classification metrics
        - metrics.accuracy_score(y_true, y_pred, *[, ...])
        - metrics.precision_score(y_true, y_pred, *[, ...])
        - metrics.recall_score(y_true, y_pred, *[, ...])
        - metrics.f1_score(y_true, y_pred, *[, ...])
        - metrics.classification_report(y_true, y_pred, *)
        - other...(https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
    '''
    
    # Accuracy score
    accuracy = accuracy_score(test, pred)
    
    # Precision score
    precision_weighted = precision_score(test, pred, average='weighted', zero_division=0)
    
    # Recall score
    recall_micro = recall_score(test, pred, average='micro', zero_division=0)
    
    # f1 score
    f1_macro = f1_score(test, pred, average='macro', zero_division=0)
    
    
    print('Accuracy:', accuracy)
    print('Precision weighted:', precision_weighted)
    print('Recall micro:', recall_micro)
    print('f1 macro', f1_macro)
    print(classification_report(y_true=test.values, 
                                y_pred=pred , 
                                target_names=target_names, zero_division=0))

In [19]:
classificationReport_cv(y_test, y_pred_test)

Accuracy: 0.14279319606087734
Precision weighted: 0.29770466261517725
Recall micro: 0.12123763418227741
f1 macro 0.040391677925941116
                        precision    recall  f1-score   support

               request       0.56      0.28      0.37      1056
                 offer       0.00      0.00      0.00        28
           aid_related       0.57      0.40      0.47      2479
          medical_help       0.12      0.00      0.00       516
      medical_products       0.25      0.00      0.01       296
     search_and_rescue       1.00      0.01      0.01       169
              security       0.00      0.00      0.00       110
              military       0.20      0.00      0.01       208
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       394
                  food       0.23      0.01      0.02       685
               shelter       0.12      0.00      0.01       558
              clothing       0.00

In [20]:
classificationReport_cv(y_train, y_pred_train)

Accuracy: 0.8120429786599015
Precision weighted: 0.9956952886505172
Recall micro: 0.8451439231150677
f1 macro 0.8430323884810322
                        precision    recall  f1-score   support

               request       0.99      0.93      0.96      3217
                 offer       1.00      0.78      0.88        73
           aid_related       0.99      0.97      0.98      7630
          medical_help       1.00      0.78      0.87      1485
      medical_products       1.00      0.75      0.86       921
     search_and_rescue       1.00      0.75      0.86       493
              security       1.00      0.72      0.84       318
              military       1.00      0.72      0.84       633
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.76      0.86      1223
                  food       1.00      0.82      0.90      2044
               shelter       0.99      0.79      0.88      1664
              clothing       0.99     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
cv.best_estimator_

Pipeline(steps=[('cvect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd97cfaf440>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=10))])

In [22]:
cv.best_params_

{'clf__max_depth': None, 'clf__min_samples_split': 2, 'clf__n_estimators': 10}

In [23]:
cv.best_score_

0.14408220924471266

### 9. Export your model as a pickle file

In [24]:
# save cv
k = pickle.dump(cv, open('model_cv.pkl', 'wb'))

In [25]:
# read cv
pickle.load(open('model_cv.pkl', 'rb'))

GridSearchCV(estimator=Pipeline(steps=[('cvect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fd97cfaf440>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             param_grid={'clf__max_depth': [5, None],
                         'clf__min_samples_split': [2, 3],
                         'clf__n_estimators': [5, 10, 50]})

In [26]:
# save pipeline
p = pickle.dump(pipeline, open('model_pipline.plk', 'wb'))

In [27]:
# read pipeline
pickle.load(open('model_pipline.plk', 'rb'))

Pipeline(steps=[('cvect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd97cfaf440>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.